In [7]:
import os

In [8]:
os.environ["OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["AZURE_OPENAI_API_KEY"] = ""

In [9]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [15]:
model_40_temp_08 = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4-turbo-2024-04-09",
    temperature=0.8,
)

output_parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "{system}"),
    ("user", "{input}")
])

chain = prompt | model_40_temp_08 | output_parser

In [16]:
system = """
    You are world class mathematician.
"""

prompt_text = """
    I need to compute 23 * 3 - 89
"""

print("System: ", system)
print("************************************************")
resp = chain.invoke({ "system": system, "input": prompt_text })
resp

System:  
    You are world class mathematician.

************************************************


'To compute the expression \\(23 \\times 3 - 89\\), start by multiplying 23 and 3:\n\n\\[23 \\times 3 = 69\\]\n\nNext, subtract 89 from 69:\n\n\\[69 - 89 = -20\\]\n\nSo, the result of \\(23 \\times 3 - 89\\) is \\(-20\\).'

In [30]:
from langchain.agents import tool


@tool
def multiply(a: int, b:int) -> int:
    """Multiplies two numbers"""
    return a*b

@tool
def substract(a: int, b:int) -> int:
    """Return the difference between two numbers, return a-b"""
    return a-b

tools = [multiply, substract]

llm_with_tools = model_40_temp_08.bind_tools(tools)

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "{system}"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
        {
            "input": lambda x: x["input"],
            "system": lambda x: x["system"],
            "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)




In [31]:
agent_executor.invoke({
    "system": "You are world class mathematician",
    "input": "I need to compute 23 * 3 - 89"
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 23, 'b': 3}`


69
Invoking: `substract` with `{'a': 69, 'b': 89}`


-20The result of the computation \(23 \times 3 - 89\) is \(-20\).

> Finished chain.


{'system': 'You are world class mathematician',
 'input': 'I need to compute 23 * 3 - 89',
 'output': 'The result of the computation \\(23 \\times 3 - 89\\) is \\(-20\\).'}